In [ ]:
!pip install emoji
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 175 kB 14.4 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=6bb07e9294f64172c9d5e92fd5e38d5315eb65ffeb6baedf9a5bdab75ad8c672
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 4.0 MB 14.3 MB/s 
     |████████████████████████████████| 6.5 MB 81.8 MB/s 
     |████████████████████████████████| 77 kB 7.4 MB/s 
     |████████████████████████████████| 895 kB 83.7 MB/s 
     |████████████████████████████████| 596 kB 69.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 16.2 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import transformers
import pandas as pd
import numpy as np
import torch
import emoji
from torch.optim import AdamW
from transformers import DebertaV2Tokenizer, DebertaV2ForMaskedLM
import sentencepiece
import re
from sklearn.model_selection import train_test_split
from transformers import pipeline
from tqdm.notebook import tqdm
RANDOM_SEED = 123

In [ ]:
torch.cuda.get_device_name(0)

'Tesla V100-SXM2-16GB'

In [ ]:
emo_data = pd.read_csv('/content/drive/MyDrive/Emogen/emogen_mask_70.csv')
emo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3071487 entries, 0 to 3071486
Data columns (total 4 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   Text           object
 1   Label          object
 2   Unmasked_Text  object
 3   Class_Text     object
dtypes: object(4)
memory usage: 93.7+ MB


In [ ]:
emo_data.head()

,Text,Label,Unmasked_Text,Class_Text
0,i hesitated to buy a pc and now it’s gone [MASK],😭,i hesitated to buy a pc and now it’s gone 😭 😭,i hesitated to buy a pc and now it’s gone
1,mouloud sghir [MASK] wtf,😭,mouloud sghir 😭 😭 😭 😭 😭 😭 😭 😭 😭 😭 😭 😭 wtf,mouloud sghir wtf
2,also nobody gonna stop saying the word himbo b...,😭,also nobody gonna stop saying the word himbo b...,also nobody gonna stop saying the word himbo b...
3,i just wanna know what channel the game gone b...,😭,i just wanna know what channel the game gone b...,i just wanna know what channel the game gone b...
4,lmfaoooooo [MASK] please seek help x,😭,lmfaoooooo 😭 😭 😭 please seek help x,lmfaoooooo please seek help x


In [ ]:
#emo_data.to_csv('/content/drive/MyDrive/Emogen/emogen_mask_70.csv', index = False)

In [ ]:
def random_mask(text):
  rand = torch.rand(len(text[0]))
  mask_arr = (rand <.12) * (text[0] != 1) * (text[0] != 0) * (text[0] != 2)
  #rand_arr = (rand >.12) * (rand < .135) * (text[0] != 1) * (text[0] != 0) * (text[0] != 2)
  selection_mask = torch.flatten((mask_arr).nonzero()).tolist()
  rand2 = torch.rand(len(selection_mask))
  rand_arr = (rand2 <.10)
  selection_rand =  torch.flatten((rand_arr).nonzero()).tolist()
  new_text = text.detach().clone()
  new_text[0, selection_mask] = 128000
  for index in selection_rand:
      rand_token  = np.random.randint(low = 10, high = len(tokenizer)-1)
      new_text[0, selection_mask[index]] = rand_token
  return new_text

In [ ]:
new_tokens = emo_data.Label.unique()
new_tokens

array(['😭', '😂', '🤣', '🙃', '😶', '🥺', '😊', '🔥', '😍', '🥰', '😅', '🤪', '😱',
       '😆', '😇', '🤔', '🤧', '😃', '😁', '😯', '😏', '🤯', '🙄', '😑', '😳', '😛',
       '🤗', '😩', '😙', '🥳', '🧐', '😉', '😔', '😘', '😒', '🤨', '🤮', '😌', '🤫',
       '😄', '😎', '😡', '😢', '\U0001f971', '😤', '🤐', '😴', '😈', '🤕', '🤤',
       '😕'], dtype=object)

In [ ]:
tokenizer = DebertaV2Tokenizer.from_pretrained('/content/drive/MyDrive/Emogen/emogen_mask_base_tokenizer')
model = DebertaV2ForMaskedLM.from_pretrained('/content/drive/MyDrive/Emogen/emogen_mask_base_debert')

In [ ]:
for token in new_tokens:
    tokenizer.add_tokens(token)
model.resize_token_embeddings(len(tokenizer))

Embedding(128016, 768, padding_idx=0)

In [ ]:
len(tokenizer)

128016

In [ ]:
class emogenDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    
    def __len__(self):
        return len(self.encodings)

    def __getitem__(self, index):
        encoded = tokenizer(self.encodings[index], return_tensors='pt', max_length=64, truncation=True, padding='max_length')
        input_ids = random_mask(encoded['input_ids'])
        labels = encoded['input_ids']
        attention_mask = encoded['attention_mask']
        token_type_ids = encoded['token_type_ids']
        return input_ids, labels, attention_mask, token_type_ids

In [ ]:
class emogen_test_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, targets):
        self.encodings = encodings
        self.targets = targets
    
    def __len__(self):
        return len(self.encodings)

    def __getitem__(self, index):
        input_ids = self.encodings[index]
        labels = self.targets[index]
        return input_ids, labels

In [ ]:
df_train, df_test = train_test_split(emo_data, test_size=0.05, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)
df_train.shape, df_val.shape, df_test.shape

((2917912, 4), (76787, 4), (76788, 4))

In [ ]:
train_dataset = emogenDataset(list(df_train.Unmasked_Text))
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=16,shuffle=True)
val_dataset = emogen_test_Dataset(list(df_val.Text), list(df_val.Label))
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=1,shuffle=True)
test_dataset = emogen_test_Dataset(list(df_test.Text), list(df_test.Label))
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=1,shuffle=True)
test_model = pipeline('fill-mask', model=model, tokenizer = tokenizer, top_k = 1, targets = new_tokens, batch_size = 1, device = 0)

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/Emogen/testing_4_5')
optimizer = AdamW(model.parameters(), lr=1.5e-5, eps = 1e-6, weight_decay = .01)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
optimizer = AdamW(model.parameters(), lr=1.5e-5, eps = 1e-6, weight_decay = .01)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()
epochs = 3
best_accuracy = 0
for epoch in range(epochs):
    loop = tqdm(train_loader)
    for input_ids, labels, attention_mask, token_type_ids in loop:
        optimizer.zero_grad()
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids= token_type_ids.to(device)
        outputs = model(input_ids.squeeze(), attention_mask=attention_mask.squeeze(), labels=labels.squeeze(), token_type_ids = token_type_ids.squeeze())
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        loop.set_description("Epoch: {}".format(epoch))
        loop.set_postfix(loss=loss.item())
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, '/content/drive/MyDrive/Emogen/testing_4_5')
    correct = []
    loop3 = tqdm(val_loader)
    for input_ids, labels in loop3:
        input_id = input_ids[0]
        if '[MASK]' in input_id:
          label = labels[0]
          output = test_model(input_id)
          predictions = output[0]['token_str']
          if predictions == label:
            correct += [1]
          else:
            correct += [0]
    if np.mean(correct) > best_accuracy:
      best_accuracy = np.mean(correct)
      model.save_pretrained("/content/drive/MyDrive/Emogen/emogen_mask_base_debert")
      tokenizer.save_pretrained("/content/drive/MyDrive/Emogen/emogen_mask_base_tokenizer")
    print('Validation Accuracy: ',np.mean(correct))

  0%|          | 0/182370 [00:00<?, ?it/s]

In [ ]:
correct = []
loop3 = tqdm(test_loader)
for input_ids, labels in loop3:
    input_id = input_ids[0]
    if '[MASK]' in input_id:
      label = labels[0]
      output = test_model(input_id)
      predictions = output[0]['token_str']
      if predictions == label:
        correct += [1]
      else:
        correct += [0]
print('Testing Accuracy: ',np.mean(correct))

  0%|          | 0/77275 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:999: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


KeyboardInterrupt: ignored

In [ ]:
test_model('Yo yo yo what up')